# VQE Deep Dive: Advanced Techniques

Explore VQE implementation details, parameter studies, and optimization strategies.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from quantum_protein_folding.models import VQEFoldingModel
from quantum_protein_folding.quantum.hamiltonian import compute_exact_ground_state
from quantum_protein_folding.analysis import analyze_convergence, compute_overlap

## 1. Ansatz Architecture Study

Compare different ansatz depths and types.

In [ ]:
sequence = "HPHPPHHPHH"

# Test different ansatz depths
depths = [1, 2, 3, 4, 5]
results = {}

for depth in depths:
    print(f"\nTesting depth={depth}...")
    
    model = VQEFoldingModel(
        sequence=sequence,
        lattice_dim=2,
        ansatz_depth=depth,
        optimizer='COBYLA'
    )
    
    result = model.run(maxiter=150)
    
    results[depth] = {
        'energy': result.optimal_value,
        'n_params': model.solver.n_params,
        'history': result.convergence_history
    }
    
    print(f"  Energy: {result.optimal_value:.4f}")
    print(f"  Parameters: {model.solver.n_params}")

In [ ]:
# Plot depth vs energy
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Energy vs depth
depths_list = list(results.keys())
energies = [results[d]['energy'] for d in depths_list]
n_params_list = [results[d]['n_params'] for d in depths_list]

ax1.plot(depths_list, energies, 'o-', linewidth=2, markersize=8, color='blue')
ax1.set_xlabel('Ansatz Depth', fontsize=12)
ax1.set_ylabel('Ground State Energy', fontsize=12)
ax1.set_title('Energy vs Ansatz Depth', fontweight='bold')
ax1.grid(True, alpha=0.3)

# Parameters vs depth
ax2.plot(depths_list, n_params_list, 's-', linewidth=2, markersize=8, color='red')
ax2.set_xlabel('Ansatz Depth', fontsize=12)
ax2.set_ylabel('Number of Parameters', fontsize=12)
ax2.set_title('Parameter Count vs Depth', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Optimizer Comparison

Compare different classical optimizers.

In [ ]:
optimizers = ['COBYLA', 'SLSQP', 'SPSA']
opt_results = {}

for opt in optimizers:
    print(f"\nTesting {opt}...")
    
    model = VQEFoldingModel(
        sequence=sequence,
        lattice_dim=2,
        ansatz_depth=3,
        optimizer=opt
    )
    
    result = model.run(maxiter=100)
    
    opt_results[opt] = {
        'energy': result.optimal_value,
        'iterations': result.n_iterations,
        'history': result.convergence_history
    }
    
    print(f"  Final energy: {result.optimal_value:.4f}")
    print(f"  Iterations: {result.n_iterations}")

In [ ]:
# Plot convergence comparison
fig, ax = plt.subplots(figsize=(10, 6))

for opt, data in opt_results.items():
    ax.plot(data['history'], label=opt, linewidth=2, alpha=0.7)

ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('Energy', fontsize=12)
ax.set_title('Optimizer Comparison', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Quantum Advantage Analysis

For small systems, compare VQE result with exact diagonalization.

In [ ]:
# Use small sequence for exact comparison
small_seq = "HPHPH"

model = VQEFoldingModel(
    sequence=small_seq,
    lattice_dim=2,
    ansatz_depth=3
)

# VQE solution
vqe_result = model.run(maxiter=100)

# Exact solution
exact_energy, exact_state = compute_exact_ground_state(model.encoding.hamiltonian)

print(f"VQE Energy: {vqe_result.optimal_value:.6f}")
print(f"Exact Energy: {exact_energy:.6f}")
print(f"Relative Error: {abs(vqe_result.optimal_value - exact_energy) / abs(exact_energy) * 100:.2f}%")

## 4. Shot Noise Analysis

Study effect of measurement shots on solution quality.

In [ ]:
shot_counts = [128, 256, 512, 1024, 2048]
shot_results = {}

for shots in shot_counts:
    print(f"\nTesting shots={shots}...")
    
    model = VQEFoldingModel(
        sequence="HPHPPHHPHH",
        lattice_dim=2,
        ansatz_depth=3,
        shots=shots
    )
    
    result = model.run(maxiter=50)
    
    shot_results[shots] = result.optimal_value
    print(f"  Energy: {result.optimal_value:.4f}")

In [ ]:
# Plot shots vs energy
fig, ax = plt.subplots(figsize=(10, 6))

shots_list = list(shot_results.keys())
energies = list(shot_results.values())

ax.semilogx(shots_list, energies, 'o-', linewidth=2, markersize=10)
ax.set_xlabel('Number of Shots', fontsize=12)
ax.set_ylabel('Final Energy', fontsize=12)
ax.set_title('Effect of Measurement Shots on Solution Quality', fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Parameter Landscape Visualization

Visualize energy landscape in parameter space (2D slice).

In [ ]:
# Create 2D parameter scan
model = VQEFoldingModel(
    sequence="HPHPH",
    lattice_dim=2,
    ansatz_depth=1  # Fewer params for visualization
)

# Get optimal parameters
result = model.run(maxiter=100)
optimal_params = result.optimal_params

# Scan around first two parameters
n_points = 30
param_range = np.linspace(-np.pi, np.pi, n_points)

energy_grid = np.zeros((n_points, n_points))

print("Computing parameter landscape...")
for i, p1 in enumerate(param_range):
    for j, p2 in enumerate(param_range):
        test_params = optimal_params.copy()
        test_params[0] = p1
        test_params[1] = p2
        
        energy_grid[i, j] = model.solver._compute_expectation_value(test_params)

# Plot
fig, ax = plt.subplots(figsize=(8, 7))

im = ax.contourf(param_range, param_range, energy_grid, levels=20, cmap='viridis')
ax.plot(optimal_params[0], optimal_params[1], 'r*', markersize=20, label='Optimum')

ax.set_xlabel(r'$\theta_0$', fontsize=12)
ax.set_ylabel(r'$\theta_1$', fontsize=12)
ax.set_title('Energy Landscape (2D Slice)', fontweight='bold')
ax.legend()

plt.colorbar(im, ax=ax, label='Energy')
plt.tight_layout()
plt.show()

## Summary

Key insights:
- Deeper ansatze provide better energy but require more parameters
- COBYLA typically performs well for small-scale problems
- 1024 shots provide good balance between accuracy and speed
- Energy landscapes can be complex with multiple local minima